In [1]:
import numpy as np
import pandas as pd
import os

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Making list of stop words , punctuation and some frequently occuring words which I observed

In [2]:
from nltk.corpus import stopwords
from string import punctuation
stop_words=stopwords.words('english')
punctuations=list(punctuation)
stop_words+=punctuations

In [3]:
Own_Stop_Words=['subject:', 'from:', 'date:', 'newsgroups:', 'message-id:', 'lines:', 'path:', 'apr', 'organization:',
 'gmt', 'would', '1993', 're:', '--', 'writes:', 'one', 'references:', 'article', '|>', 'sender:', 'like', 'nntp-posting-host:',
 'people', 'get', 'university', '93', 'know', '>>', 'think', 'may', 'use', 'also', 'x', 'new', 'xref:', 'cantaloupe.srv.cs.cmu.edu',
 'could', "i'm","max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'"]

In [4]:
stop_words+=Own_Stop_Words

### Making list of newsgroups

In [5]:
folders=['alt.atheism' , 'comp.graphics' , 'comp.os.ms-windows.misc' , 'comp.sys.ibm.pc.hardware' ,
            'comp.sys.mac.hardware' , 'comp.windows.x' , 'misc.forsale' , 'rec.autos' , 'rec.motorcycles' ,
            'rec.sport.baseball' , 'rec.sport.hockey' , 'sci.crypt' , 'sci.electronics' , 'sci.med' ,
            'sci.space' , 'soc.religion.christian' , 'talk.politics.guns' , 'talk.politics.mideast' , 
            'talk.politics.misc' , 'talk.religion.misc']

In [6]:
folders[2]

'comp.os.ms-windows.misc'

### Loading data into kernel 

In [7]:
DATA_DIR='20_newsgroups/'

In [8]:
data={} # data is a dictionary of the form { subfolder : [text1,text2,....,text1000] }
for subfolder in folders:
    data[subfolder]=[]
    files=os.listdir(os.path.join(DATA_DIR,subfolder))
    for file in files:
        with open(os.path.join(DATA_DIR,subfolder,file),encoding="ISO-8859-1") as opened_file:
            data[subfolder].append(opened_file.read())

In [ ]:
# os.listdir('20_newsgroups/alt.atheism')

In [ ]:
len(data[folders[19]])

In [9]:
# Create a dictionary of words with frequency
vocab={}
for i in range(len(data)): # for each newsgroup
    for doc in data[folders[i]]: # for each item in the list of current newsgroup
        # Split each document into words now
        for word in doc.split(): # doc.split() will give us a list of words, we iterate over each word of that list 
            if word.lower() not in stop_words and len(word.lower()) >= 5:
                if word.lower() not in vocab:
                    vocab[word.lower()]=1
                else:
                    vocab[word.lower()]+=1

In [10]:
len(data[folders[0]])

1000

In [11]:
# Sort the dictionary based on values
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1),reverse=True)

In [12]:
len(sorted_vocab)

390180

In [18]:
sorted_vocab_list=[]
for pair in sorted_vocab:
    sorted_vocab_list.append(pair[0])
# Select top 2000 features from this document
feature_list=sorted_vocab_list[0:2000]
len(feature_list)

2000

In [43]:
Y=[]
docs=[]
for subfolder in folders:
    files=os.listdir(os.path.join(DATA_DIR,subfolder))
    for file in files:
        Y.append(subfolder)
        with open(os.path.join(DATA_DIR,subfolder,file),encoding='latin-1') as opened_file:
            docs.append(opened_file.read())
len(Y) # religion file has 997 items while every other has 1000 items each
Y=np.array(Y)
Y.shape

(19997,)

In [ ]:
X = np.zeros((len(docs),len(feature_list)))
for row in range(len(docs)):
    for word in docs[row]:
        for feature in feature_list:
            if word==feature:
                X[row][feature]+=1

## Implementing Multinomial NaiveBayes

In [45]:
def fit(X_train,Y_train):
    result={}
    class_labels=set(Y_train)
    result["total_data"]=len(X_train)
    for current_label in class_labels:
        result[current_label]={}
        current_rows=(Y_train==current_label)
        x_train_current=X_train[current_rows]
        y_train_current=Y_train[current_rows]
        total_count=0
        for j in len(feature_list):
            result[current_label][feature_list[j]]=x_train_current[:,j].sum()
            total_count+=x_train_current[:,j].sum()
        result["total_count"]=total_count
    return result

In [50]:
def probability(x,dictionary,current_class):
    output=np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    for i in range(feature_list):
        current_word_count=dictionary[current_class][feature_list[i]]+1
        total_word_count=dictionary[current_class]["total_count"]+len(feature_list)
        current_word_probability=np.log(current_word_count)-np.log(total_word_count)
        for j in range(int(row[i])): # if the frequency of word in test data point is zero then we wont consider it.
            output+=current_word_probability
    return output

In [47]:
def predictSingleClass(x,dictionary):
    best_class=-1000
    best_prob=-1000
    firstRun=True
    possible_classes=len(dictionary.keys())-1
    for current_class in possible_classes:
        if current_class=="total_data":
            continue
        current_class_probability=probability(x,dictionary,current_class)
        if(firstRun==True or current_class_probability>best_prob):
            best_class=current_class
            best_prob=current_class_probability
        firstRun=False
    return best_prob

In [46]:
def predict(X_test,dictionary):
    Y_pred=[]
    for x in X_test:
        Y_pred.append(predictSingleClass(x,dictionary))
    return Y_pred